# Produce Movie Reviews with Positive Sentiment

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CarperAI/trlx/blob/main/examples/notebooks/trlx_sentiments.ipynb)

#### Optimize gpt2 to review movies positively based on a corpus of IMDB reviews with sentiment scores from DistilBert.

Notebook by [@zswitten](https://github.com/zswitten)

---

Execute the cells below to install [TRLX](https://github.com/CarperAI/trlx) for a colab environment.

In [ ]:
!git clone https://github.com/CarperAI/trlx.git
!git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

Cloning into 'trlx'...
remote: Enumerating objects: 5140, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 5140 (delta 93), reused 124 (delta 69), pack-reused 4970
Receiving objects: 100% (5140/5140), 46.17 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (3228/3228), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.9/749.9 KB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [ ]:
# uninstall scikit_learn + jax to avoid numpy issues
!pip uninstall -y scikit_learn jax

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Found existing installation: jax 0.3.25
Uninstalling jax-0.3.25:
  Successfully uninstalled jax-0.3.25


In [ ]:
import os

# run within repo
os.chdir('/content/trlx')
print(os.getcwd())

/content/trlx


In [ ]:
import yaml
from datasets import load_dataset
from transformers import pipeline
import pathlib
from typing import Dict, List
import trlx
from trlx.data.default_configs import TRLConfig, default_ilql_config

In [ ]:
default_config = default_ilql_config().to_dict()
default_config['train']['tracker'] = None
default_config['train']['batch_size'] = 16
default_config['train']['epochs'] = 10
config = TRLConfig.update(default_config, {})
print(config)

{
    "method": {
        "name": "ilqlconfig",
        "tau": 0.7,
        "gamma": 0.99,
        "cql_scale": 0.1,
        "awac_scale": 1,
        "alpha": 0.001,
        "beta": 0,
        "steps_for_target_q_sync": 5,
        "two_qs": true,
        "gen_kwargs": {
            "max_new_tokens": 56,
            "top_k": 20,
            "beta": 4,
            "temperature": 1.0
        }
    },
    "model": {
        "model_path": "gpt2",
        "model_arch_type": "causal",
        "num_layers_unfrozen": -1,
        "delta_kwargs": null
    },
    "optimizer": {
        "name": "adamw",
        "kwargs": {
            "lr": 5e-05,
            "betas": [
                0.9,
                0.95
            ],
            "eps": 1e-08,
            "weight_decay": 1e-06
        }
    },
    "scheduler": {
        "name": "cosine_annealing",
        "kwargs": {
            "T_max": 1000,
            "eta_min": 5e-05
        }
    },
    "tokenizer": {
        "tokenizer_path": "gpt2",

In [ ]:
def get_positive_score(scores):
    "Extract value associated with a positive sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]

sentiment_fn = pipeline(
    "sentiment-analysis",
    "lvwerra/distilbert-imdb",
    top_k=2,
    truncation=True,
    batch_size=256,
    device=0,
)

def metric_fn(samples: List[str], **kwargs) -> Dict[str, List[float]]:
    sentiments = list(map(get_positive_score, sentiment_fn(samples)))
    return {"sentiments": sentiments}

imdb = load_dataset("imdb", split="train+test")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


In [ ]:
trainer = trlx.train(
    samples=imdb["text"], 
    rewards=imdb["label"],
    eval_prompts=[
        "I don't know much about Hungarian underground",
        "What made this movie so distinctly",
        "Like the sandwich I just bought at the grocery store,",
        "I cannot believe how much this movie made me want to"
    ] * 20,
    metric_fn=metric_fn,
    config=config,
)

[RANK 0] Initializing model: gpt2


[RANK 0] Collecting rollouts
Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 1024). Running this sequence through the model will result in indexing errors
[RANK 0] Logging sample example


                                                  Sample Example                                                   
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Prompt        ┃ Response                                                                               ┃ Reward ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ <|endoftext|> │ I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that   │ 0      │
│               │ surrounded it when it was first released in 1967. I also heard that at first it was    │        │
│               │ seized by U.S. customs if it ever tried to enter this country, therefore being a fan   │        │
│               │ of films considered "                                                                  │        │
└───────────────┴────────────────────────────────────────────────────────────────────────────────────────┴────────┘

[RANK 0] Logging experience string statistics


     Experience String Stats (mean ∈ [min, max])      
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt Length ┃ Output Length   ┃ Sample Length    ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 1.00 ∈ [1, 1] │ 62.76 ∈ [9, 63] │ 63.76 ∈ [10, 64] │
└───────────────┴─────────────────┴──────────────────┘

...

                                     Evaluation #10 metrics/sentiments: 0.507                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                                        ┃ output                                             ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ I don't know much about Hungarian underground │  cinema but when i saw the film I was very         │ 0.992      │
│                                               │ interested to try it as an experience and as it    │            │
│                                               │ was very low budget then i was very glad to come   │            │
│                                               │ across it. I was very pleasantly shocked and       │            │
│                                               │ impressed with the film and the cast. The story of │            │
│                                               │ the group is very interesting                      │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ What made this movie so distinctly            │  special was the fact it was made by an all time   │ 0.979      │
│                                               │ best known actor as Michael Caine (I was born in   │            │
│                                               │ the '80s). His role of the great character in the  │            │
│                                               │ film was an inspiration for the film that made me  │            │
│                                               │ want to be a singer, but the story and             │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ Like the sandwich I just bought at the        │  local store I must say the "pigeon pigeons" in    │ 0.372      │
│                                               │ this movie are not the same as the "pigeons", but  │            │
│                                               │ the same "pigeons". The whole plot is similar to   │            │
│                                               │ the "Pigeon Stag" though the whole plot is         │            │
│                                               │ different                                          │            │
└───────────────────────────────────────────────┴────────────────────────────────────────────────────┴────────────┘

[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/5]:   0%|          | 0/5 [00:00<?, ?it/s]

[RANK 0] Computing metrics
[RANK 0] Summarizing evaluation


                                     Evaluation #11 metrics/sentiments: 0.612                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                                        ┃ output                                             ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ I don't know much about Hungarian underground │ -re-en and underground-in, but if the plot was set │ 0.635      │
│                                               │ in real life, then, just as for the main           │            │
│                                               │ character, the story just moves on, the plot just  │            │
│                                               │ moves on. There are very few interesting scenes of │            │
│                                               │ this one and all of the other films are            │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ What made this movie so distinctly            │  and really, really bad and I just wanted to       │ 0.0379     │
│                                               │ comment. The fact I'm in this movie is really,     │            │
│                                               │ really, really, really, really stupid. This movie, │            │
│                                               │ it really makes me, really, truly, really, really, │            │
│                                               │ really, really, really, really, really             │            │
├───────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────┤
│ Like the sandwich I just bought at the        │  local store, I still remember the original one.   │ 0.981      │
│                                               │ The story has all the elements of the original: a  │            │
│                                               │ young man, an old man, the daughter of a man who   │            │
│                                               │ has been living with the boy for the last five     │            │
│                                               │ years, his daughter, her mother and her husband's  │            │
│                                               │ best                                               │            │
└───────────────────────────────────────────────┴────────────────────────────────────────────────────┴────────────┘

In [ ]:
# output
input_str = 'One thing you should know about When Sally Met Harry is that'
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output))

One thing you should know about When Sally Met Harry is that there's so much more she's done than in this one. She's got more to offer and more to say for a long, long time. She's still very much a mystery in her own right and the movie does a good job of the plot.<
